In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110851 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
--2019-01-24 14:34:33--  https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
Resolving launchpad.net (launchpad.net)... 91.189.89.223, 91.189.89.222, 2001:67c:1560:8003::8004, ...
Connecting to launchpad.net (launchpad.net)|91.189.89.223|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2019-01-24 14:34:34 ERROR 404: Not Found.

dpkg: error: cannot access archive 'google-drive-ocaml

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import tensorflow as tf
from sklearn.datasets import load_files
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2

In [0]:
def load_data(folder_path):
  data = load_files(folder_path)
  files = np.array(data['filenames'])
  targ = np.array(data['target'])
  lb = LabelBinarizer()
  targ = lb.fit_transform(targ)
  return files,targ
  

In [0]:
fold = 'drive/datasets/flowers'

In [0]:
img_files,targets = load_data(fold)

In [0]:
def extract(img_file):
  img = plt.imread(img_file)
  #img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  img = cv2.resize(img,(200,200))
  return img

In [0]:
def tensor_4d(fil):
  list_of_tensors = [extract(im) for im in tqdm(fil)]
  return np.stack(list_of_tensors,axis=0)

In [9]:
imgs = tensor_4d(img_files)

100%|██████████| 4323/4323 [00:26<00:00, 160.74it/s]


In [0]:
inp = tf.placeholder(dtype='float32',shape=[None,200,200,3],name='inputs')
lab = tf.placeholder(dtype='float32',shape=[None,5],name='labels')

In [0]:
def resnet(inp,input_shape):
  res = tf.keras.applications.ResNet50(weights='imagenet',input_shape=input_shape,include_top=False)
  custom = res(inp)
  custom = tf.keras.layers.Flatten()(custom)
  custom = tf.keras.layers.Dense(units=500,activation='relu')(custom)
  custom = tf.keras.layers.Dropout(0.2)(custom)
  custom = tf.keras.layers.Dense(units=100,activation='relu')(custom)
  custom = tf.keras.layers.Dropout(0.2)(custom)
  custom = tf.keras.layers.Dense(5)(custom)
  return custom

In [12]:
logits = resnet(inp,(200,200,3))
fin_pre = tf.nn.softmax(logits)
preds = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=lab)
cost = tf.reduce_mean(preds)
opt = tf.train.AdamOptimizer(learning_rate=0.001)
train = opt.minimize(cost)
#correct_pred = tf.equal(tf.argmax(preds,1),tf.argmax(lab,1))
#acc = tf.get_variable("accur", [1.0])
#acc = tf.reduce_mean(tf.cast(correct_pred,dtype='float32'))
acc,acc_op = tf.metrics.accuracy(labels=tf.argmax(lab,1),predictions=tf.argmax(fin_pre,1))
init = tf.global_variables_initializer()

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 3s 0us/step


In [0]:
with tf.Session() as sess:
  sess.run(init)
  sess.run(tf.local_variables_initializer())
  for epoch in range(50):
    batches = len(imgs)//32
    for start,end in zip(range(0,len(imgs),32),range(32,len(imgs),32)):
      xtr = imgs[start:end]
      ytr = targets[start:end]
      print(ytr.shape)
      #print(end)
      sess.run(train,feed_dict={inp:xtr,lab:ytr})
      sess.run([acc,acc_op],feed_dict={inp:xtr,lab:ytr})
      print('Accuracy:',sess.run([acc],feed_dict={inp:xtr,lab:ytr}))
      
  

(32, 5)
Accuracy: [0.21875]
(32, 5)
Accuracy: [0.171875]
(32, 5)
Accuracy: [0.16666667]
(32, 5)
Accuracy: [0.203125]
(32, 5)
Accuracy: [0.2]
(32, 5)
Accuracy: [0.203125]
(32, 5)
Accuracy: [0.20982143]
(32, 5)
Accuracy: [0.21484375]
(32, 5)
Accuracy: [0.20833333]
(32, 5)
Accuracy: [0.196875]
(32, 5)
Accuracy: [0.1903409]
(32, 5)
Accuracy: [0.19010417]
(32, 5)
Accuracy: [0.20192307]
(32, 5)
Accuracy: [0.19419643]
(32, 5)
Accuracy: [0.18541667]
(32, 5)
Accuracy: [0.18554688]
(32, 5)
Accuracy: [0.18566176]
(32, 5)
Accuracy: [0.19791667]
(32, 5)
Accuracy: [0.19736843]
(32, 5)
Accuracy: [0.196875]
(32, 5)
Accuracy: [0.19642857]
(32, 5)
Accuracy: [0.19602273]
(32, 5)
Accuracy: [0.19565217]
(32, 5)
Accuracy: [0.19661458]
(32, 5)
Accuracy: [0.2]
(32, 5)
Accuracy: [0.203125]
(32, 5)
Accuracy: [0.20601852]
(32, 5)
Accuracy: [0.20647322]
(32, 5)
Accuracy: [0.20581897]
(32, 5)
Accuracy: [0.203125]
(32, 5)
Accuracy: [0.20362903]
(32, 5)
Accuracy: [0.19726562]
(32, 5)
Accuracy: [0.19507575]
(32, 5)
A

ValueError: ignored